In [534]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pickle

In [400]:
# Chrome Driver 104
DRIVER_PATH = '/Users/edwardhuang/Documents/interview/Dave And Busters/chromedriver'

# Use Chrome Beta, compatible with above Chrome Driver
options = Options()
#options.headless = True
options.binary_location = '/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta'

In [511]:
# Set up the driver
driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
driver.implicitly_wait(30)

# Launch DnB Homepage
driver.get('https://www.daveandbusters.com')

# Launch DnB Locations Page
locationPage = WebDriverWait(driver,10).until(
    EC.presence_of_element_located(
        (By.PARTIAL_LINK_TEXT,'Location'),
    )
).click()

<ipython-input-511-79e19c123114>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)


In [462]:
# Cache all the location links v1
address1List = []
address2List = []
address3List = []
locationList = []

fullLocTable = driver.find_element(By.ID, 'dnb-locations-list')
fullLocTableExMap = fullLocTable.find_element(By.CSS_SELECTOR,'div[class="row"]')
locCols = fullLocTableExMap.find_elements(By.CSS_SELECTOR,'div[class="col-md-3"]')
for colIdx in range(len(locCols)):
    locCols = getColumns()
    states = locCols[colIdx].find_elements(By.CSS_SELECTOR,'div[class="location-list-item"]')
    for stateIdx in range(len(states)):
        locations = states[stateIdx].find_element(By.TAG_NAME,'ul')
        locations = locations.find_elements(By.CSS_SELECTOR,'li[class="dnb-location-nowopen-bullet"]')
        for locIdx in range(len(locations)):
            location = locations[locIdx]
            locationName = location.find_element(By.TAG_NAME, 'a').get_attribute('innerHTML').strip()
            locationLink = location.find_element(By.TAG_NAME,'a').get_attribute('href')
            try:
                locationList.append(locationLink)
            except:
                print("Couldn't pull ", locationName)


In [517]:
# Get the Address v1
for location in locationLink:
    driver.get(location)

    # Find the table with the address
    bigTable = driver.find_element(By.CSS_SELECTOR,'div[class="location-header-details-inner"]')
    # Filter down into the address
    addresses = bigTable.find_elements(By.CSS_SELECTOR,'div[class="location-address ng-binding"]')
    # Filter down into the city state
    citystate = bigTable.find_element(By.CSS_SELECTOR,'div[class="location-address-2 ng-binding"]')

    # Append the data to a list
    address1List.append(addresses[0].get_attribute('innerHTML'))
    address2List.append(addresses[1].get_attribute('innerHTML'))
    address3List.append(citystate.get_attribute('innerHTML'))

            

2700 Riverchase Galleria


In [505]:
# Put info into a df
df = pd.DataFrame(data = {
    'Address 1': address1List,
    'Address 2': address2List,
    'City State Zip': address3List
})

# TO DO: city, state, zip
city, state, zipcode = df['City State Zip'].str.split(", ",expand=True)

# Create full address str
df['Full Address'] = df['Address 1'] + ' ' + df['Address 2'] + ' ' + df['City State Zip']


,Address 1,Address 2,City State Zip,Full Address
0,2700 Riverchase Galleria,Ste 110,"Birmingham, AL, 35244","2700 Riverchase Galleria Ste 110 Birmingham, A..."
1,950 Makers Way NW,,"Huntsville, AL, 35806","950 Makers Way NW Huntsville, AL, 35806"
2,800 East Dimond Blvd.,Suite 240,"Anchorage, AK, 99515","800 East Dimond Blvd. Suite 240 Anchorage, AK,..."
3,9460 West Hanna Lane,,"Glendale, AZ, 85305","9460 West Hanna Lane Glendale, AZ, 85305"
4,21001 N. Tatum Blvd.,Suite 44-1400,"Phoenix, AZ, 85050","21001 N. Tatum Blvd. Suite 44-1400 Phoenix, AZ..."
...,...,...,...,...
143,1101 Outlet Collection Drive SW,,"Auburn, WA, 98001","1101 Outlet Collection Drive SW Auburn, WA, 9..."
144,11639 NE 4th Street,,"Bellevue, WA, 98004","11639 NE 4th Street Bellevue, WA, 98004"
145,202 Bay Park Square,,"Green Bay, WI, 54304","202 Bay Park Square Green Bay, WI, 54304"
146,414 West Towne Mall,#C,"Madison, WI, 53719","414 West Towne Mall #C Madison, WI, 53719"


In [ ]:
#df.to_pickle('dnb_addresses')
#df = pd.read_pickle('dnb_addresses')

In [ ]:
# Google Review v1
n = 1
reviewList = []

for store in df['Full Address']:    
    driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
    driver.implicitly_wait(30)
    driver.get('https://www.google.com')

    # Identify search box
    m = driver.find_element(By.NAME,"q")
    # Enter search text
    m.send_keys('Dave and Busters ' + store)
    time.sleep(0.2)
    # Perform Google search
    m.send_keys(Keys.ENTER)

    time.sleep(7) # Bad practice? use wait.until?

    # I used class name this time because it seems more generic than xpath. 
    # There might be an even better search element.
    review = driver.find_element(
        By.CLASS_NAME,'Aq14fc').text
    
    reviewList.append(review)
    print(review)
            
driver.close()

In [196]:
df['Reviews'] = reviewList
df

,Address 1,Address 2,City State Zip,Full Address,Reviews
0,2700 Riverchase Galleria,Ste 110,"Birmingham, AL, 35244","2700 Riverchase Galleria Ste 110 Birmingham, A...",4.1
1,950 Makers Way NW,,"Huntsville, AL, 35806","950 Makers Way NW Huntsville, AL, 35806",4.3
2,800 East Dimond Blvd.,Suite 240,"Anchorage, AK, 99515","800 East Dimond Blvd. Suite 240 Anchorage, AK,...",4.0
3,200 Premium Outlets Dr,,"Blackwood, NJ, 08012","200 Premium Outlets Dr Blackwood, NJ, 08012",4.3
4,310 Willowbrook Mall,,"Wayne, NJ, 07470","310 Willowbrook Mall Wayne, NJ, 07470",4.1
5,274 Woodbridge Center Dr,,"Woodbridge, NJ, 07095","274 Woodbridge Center Dr Woodbridge, NJ, 07095",4.1
